<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500 height=450/></p>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Продвинутый поток. Весна 2021</b></h3>

Это домашнее задание будет посвящено полноценному решению задачи машинного обучения.

Есть две части этого домашнего задания: 
* Сделать полноценный отчет о вашей работе: как вы обработали данные, какие модели попробовали и какие результаты получились (максимум 10 баллов). За каждую выполненную часть будет начислено определенное количество баллов.
* Лучшее решение отправить в соревнование на [kaggle](https://www.kaggle.com/c/advanced-dls-spring-2021/) (максимум 5 баллов). За прохождение определенного порогов будут начисляться баллы.


**Обе части будут проверяться в формате peer-review. Т.е. вашу посылку на степик будут проверять несколько других студентов и аггрегация их оценок будет выставлена. В то же время вам тоже нужно будет проверить несколько других учеников.**

**Пожалуйста, делайте свою работу чистой и понятной, чтобы облегчить проверку. Если у вас будут проблемы с решением или хочется совета, то пишите в наш чат в телеграме или в лс @runfme. Если вы захотите проаппелировать оценку, то пипшите в лс @runfme.**

**Во всех пунктах указания это минимальный набор вещей, которые стоит сделать. Если вы можете сделать какой-то шаг лучше или добавить что-то свое - дерзайте!**

# Как проверять?

Ставьте полный балл, если выполнены все рекомендации или сделано что-то более интересное и сложное. За каждый отсустствующий пункт из рекомендация снижайте 1 балл.

# Метрика

Перед решением любой задачи важно понимать, как будет оцениваться ваше решение. В данном случае мы используем стандартную для задачи классификации метрику ROC-AUC. Ее можно вычислить используя только предсказанные вероятности и истинные классы без конкретного порога классификации + она раотает даже если классы в данных сильно несбалансированны (примеров одного класса в десятки раз больше примеров длугого). Именно поэтому она очень удобна для соревнований.

Посчитать ее легко:


In [ ]:
from sklearn.metrics import roc_auc_score

y_true = [
    0,
    1,
    1,
    0,
    1
]

y_predictions = [
    0.1,
    0.9,
    0.4,
    0.6,
    0.61
]

roc_auc_score(y_true, y_predictions)

# Первая часть. Исследование

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

## Загрузка данных (2 балла)

1) Посмотрите на случайные строчки. 

2) Посмотрите, есть ли в датасете незаполненные значения (nan'ы) с помощью data.isna() или data.info() и, если нужно, замените их на что-то. Будет хорошо, если вы построите табличку с количеством nan в каждой колонке.

In [ ]:
data = pd.read_csv('./train.csv')

In [ ]:
# Для вашего удобства списки с именами разных колонок

# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'
data.head()

In [ ]:
print(data.info())
print(data.isnull().sum())
print(data.isin([' ']).sum())
print(data.isin(['?']).sum())

In [ ]:
# Проверка на признаки из одного элемента
for col in cat_cols:
    print(f"{col} DISTRIBUTION")
    print(data[col].value_counts())
    print()
    
for col in num_cols:
    print(f"{col} DISTRIBUTION")
    print(data[col].value_counts())
    print()

In [ ]:
# Т.к. всего 9 строк, в которых пробел, можно удалить их, на результат это сильно не повлияет.
data['TotalSpent'] = data['TotalSpent'].replace(' ', np.nan)
data['TotalSpent'] = data['TotalSpent'].astype(float)
data = data.dropna()

## Анализ данных (3 балла)

1) Для численных призанков постройте гистограмму (*plt.hist(...)*) или boxplot (*plt.boxplot(...)*). Для категориальных посчитайте количество каждого значения для каждого признака. Для каждой колонки надо сделать *data.value_counts()* и построить bar диаграммы *plt.bar(...)* или круговые диаграммы *plt.pie(...)* (хорошо, елси вы сможете это сделать на одном гарфике с помощью *plt.subplots(...)*). 

2) Посмотрите на распределение целевой переменной и скажите, являются ли классы несбалансированными.

3) (Если будет желание) Поиграйте с разными библиотеками для визуализации - *sns*, *pandas_visual_analysis*, etc.

Второй пункт очень важен, потому что существуют задачи классификации с несбалансированными классами. Например, это может значить, что в датасете намного больше примеров 0 класса. В таких случаях нужно 1) не использовать accuracy как метрику 2) использовать методы борьбы с imbalanced dataset (обычно если датасет сильно несбалансирован, т.е. класса 1 в 20 раз меньше класса 0).

In [ ]:
data.hist(column=num_cols+[target_col], figsize=(14, 10))

In [ ]:
fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize=(15, 15))
for i in range(16):
    axes[i // 4][i % 4].set(title=cat_cols[i])
    axes[i // 4][i % 4].pie(data[cat_cols[i]].value_counts())

plt.show()

(Дополнительно) Если вы нашли какие-то ошибки в данных или выбросы, то можете их убрать. Тут можно поэксперементировать с обработкой данных как угодно, но не за баллы.

In [ ]:
data.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

## Применение линейных моделей (3 балла)

1) Обработайте данные для того, чтобы к ним можно было применить LogisticRegression. Т.е. отнормируйте числовые признаки, а категориальные закодируйте с помощью one-hot-encoding'а. 

2) С помощью кроссвалидации или разделения на train/valid выборку протестируйте разные значения гиперпараметра C и выберите лучший (можно тестировать С=100, 10, 1, 0.1, 0.01, 0.001) по метрике ROC-AUC. 

Если вы разделяете на train/valid, то используйте LogisticRegressionCV. Он сам при вызове .fit() подберет параметр С. (не забудьте передать scroing='roc_auc', чтобы при кроссвалидации сравнивались значения этой метрики, и refit=True, чтобы при потом модель обучилась на всем датасете с лучшим параметром C). 


(более сложный вариант) Если вы будете использовать кроссвалидацию, то преобразования данных и LogisticRegression нужно соединить в один Pipeline с помощью make_pipeline, как это делалось во втором семинаре. Потом pipeline надо передать в GridSearchCV. Для one-hot-encoding'a можно испльзовать комбинацию LabelEncoder + OneHotEncoder (сначала превращаем строчки в числа, а потом числа првращаем в one-hot вектора.)

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

In [ ]:
dummy_features = pd.get_dummies(data[cat_cols])
X = pd.concat([data[num_cols], dummy_features], axis=1)
y = data[target_col]

In [ ]:
lg = make_pipeline(StandardScaler(),
                  LogisticRegression())
print(lg.get_params().keys())
parametrs = {'logisticregression__C' : [100, 10, 1, 0.1, 0.01, 0.001]}
lg_grid_search = GridSearchCV(estimator=lg, 
                               param_grid=parametrs,
                               cv=5,
                               scoring='roc_auc',
                               n_jobs=-1, 
                               verbose=10,
                               refit=True)

lg_grid_search.fit(X, y)

Выпишите какое лучшее качество и с какими параметрами вам удалось получить

In [ ]:
from sklearn.metrics import roc_auc_score
print(lg_grid_search.best_params_)
print(lg_grid_search.best_score_)
print(roc_auc_score(y, lg_grid_search.predict_proba(X).T[1]))

## Применение градиентного бустинга (2 балла)

Если вы хотите получить баллы за точный ответ, то стоит попробовать градиентный бустинг. Часто градиентный бустинг с дефолтными параметрами даст вам 80% результата за 0% усилий.

Мы будем использовать catboost, поэтому нам не надо кодировать категориальные признаки. catboost сделает это сам (в .fit() надо передать cat_features=cat_cols). А численные признаки нормировать для моделей, основанных на деревьях не нужно.

1) Разделите выборку на train/valid. Протестируйте catboost cо стандартными параметрами.

2) Протестируйте разные занчения параметроа количества деревьев и learning_rate'а и выберите лучшую по метрике ROC-AUC комбинацию. 

(Дополнительно) Есть некоторые сложности с тем, чтобы использовать CatBoostClassifier вместе с GridSearchCV, поэтому мы не просим использовать кроссвалидацию. Но можете попробовать)

In [ ]:
# !pip install catboost
import catboost

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[cat_cols + num_cols], y.values, 
                                                       train_size=0.8, 
                                                       random_state=42)


In [ ]:
res = []
num = []
for i in range(50):
  boosting_model = catboost.CatBoostClassifier(n_estimators=200, 
                                              cat_features=cat_cols, 
                                              learning_rate=0.02 * i + 0.02,
                                              verbose=0)

  boosting_model.fit(X_train, y_train)

  y_train_predicted = boosting_model.predict_proba(X_train)[:, 1]
  y_test_predicted = boosting_model.predict_proba(X_test)[:, 1]
  train_auc = roc_auc_score(y_train, y_train_predicted)
  test_auc = roc_auc_score(y_test, y_test_predicted)
  res.append(roc_auc_score(y_test, y_test_predicted))
  num.append(0.02 + 0.02 * i)
  # print(train_auc, test_auc)

In [ ]:
res

In [ ]:
plt.plot(num, res) # Приблизительно определяю лучший learning rate

In [ ]:
res = []
num = []
for i in range(50):
  boosting_model = catboost.CatBoostClassifier(n_estimators=500, 
                                              cat_features=cat_cols, 
                                              learning_rate=0.004 * i + 0.004,
                                              verbose=0)

  boosting_model.fit(X_train, y_train)

  y_train_predicted = boosting_model.predict_proba(X_train)[:, 1]
  y_test_predicted = boosting_model.predict_proba(X_test)[:, 1]
  train_auc = roc_auc_score(y_train, y_train_predicted)
  test_auc = roc_auc_score(y_test, y_test_predicted)
  res.append(roc_auc_score(y_test, y_test_predicted))
  num.append(0.004 + 0.004 * i)
  print(train_auc, test_auc)

In [ ]:
plt.plot(num, res) # Чуть точнее определяю лучший learning rate

In [ ]:
res = []
num = []
for i in range(50):
  boosting_model = catboost.CatBoostClassifier(n_estimators=500, 
                                              cat_features=cat_cols, 
                                              learning_rate=0.0005 * i + 0.0005)

  boosting_model.fit(X_train, y_train)

  y_train_predicted = boosting_model.predict_proba(X_train)[:, 1]
  y_test_predicted = boosting_model.predict_proba(X_test)[:, 1]
  train_auc = roc_auc_score(y_train, y_train_predicted)
  test_auc = roc_auc_score(y_test, y_test_predicted)
  res.append(roc_auc_score(y_test, y_test_predicted))
  num.append(0.0005 + 0.0005 * i)
  print(train_auc, test_auc)

In [ ]:
plt.plot(num, res) # Чуть точнее определяю лучший learning rate

In [ ]:
res = []
num = []
for i in range(50):
  boosting_model = catboost.CatBoostClassifier(n_estimators=500, 
                                              cat_features=cat_cols, 
                                              learning_rate=0.021 * i + 0.0001)

  boosting_model.fit(X_train, y_train)

  y_train_predicted = boosting_model.predict_proba(X_train)[:, 1]
  y_test_predicted = boosting_model.predict_proba(X_test)[:, 1]
  train_auc = roc_auc_score(y_train, y_train_predicted)
  test_auc = roc_auc_score(y_test, y_test_predicted)
  res.append(roc_auc_score(y_test, y_test_predicted))
  num.append(0.021 + i * 0.0001)
  print(train_auc, test_auc)

In [ ]:
num = [0.021 + i * 0.0001 for i in range(50)]
print(res)
plt.plot(num, res) # Определяю лучший learning rate

Выпишите какое лучшее качество и с какими параметрами вам удалось получить


leraning rate = 0.022
roc_auc_score(test) = 0.8427

# Предсказания

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
def compute_meta_feature(model, X_train, X_test, y_train, cv):
    try:
        train_answers = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
        model.fit(X_train, y_train)
        return train_answers, model.predict_proba(X_test)[:, 1]
    
    except Exception:
        train_answers = cross_val_predict(model, X_train, y_train, cv=cv, method='predict')[:, 1]
        model.fit(X_train, y_train)
        return train_answers, model.predict(X_test)[:, 1]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from tqdm.notebook import tqdm

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                    train_size=0.8,
                                                    random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
models = []
#models.append(KNeighborsClassifier(n_jobs=-1, n_neighbors=30))
models.append(LogisticRegression(C=100)) # Использую значенип, полученное в пункте про лин. модели.
models.append(RandomForestClassifier(max_depth=3, n_estimators=50, n_jobs=-1))
models.append(RandomForestClassifier(max_depth=7, n_estimators=50, n_jobs=-1))
models.append(DecisionTreeClassifier(max_depth=8))

In [ ]:
meta_features_train = np.zeros((X_train.shape[0], 0))
meta_features_test = np.zeros((X_test.shape[0], 0))

In [ ]:
for model in tqdm(models):
    train, test = compute_meta_feature(model, X_train, X_test, y_train, 4)
    meta_features_train = np.append(meta_features_train, train.reshape((train.size, 1)), axis=1)
    meta_features_test = np.append(meta_features_test, test.reshape((test.size, 1)), axis=1)

In [ ]:
stacking_model = LogisticRegression(C=100)
stacking_model.fit(meta_features_train, y_train)


y_train_predicted = stacking_model.predict_proba(meta_features_train)[:, 1]
y_test_predicted = stacking_model.predict_proba(meta_features_test)[:, 1]

train_auc = roc_auc_score(y_train, y_train_predicted)
test_auc = roc_auc_score(y_test, y_test_predicted)
print(train_auc, test_auc)

**Посмотрю различные модели и выберу среди них лучшую.**

In [ ]:
# Обычная логистическая регрессия.
dummy_features = pd.get_dummies(data[cat_cols])
X_train = pd.concat([data[num_cols], dummy_features], axis=1)
y_train = data[target_col]
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)

bs1 = LogisticRegression(C=100)
bs1.fit(X_train, y_train)

In [ ]:
X_test = pd.read_csv('./test.csv')
X_test['TotalSpent'] = X_test['TotalSpent'].replace(' ', 0)
X_test['TotalSpent'] = X_test['TotalSpent'].astype(float)
dummy_features = pd.get_dummies(X_test[cat_cols])
X_test = pd.concat([X_test[num_cols], dummy_features], axis=1)
scaler.transform(X_test)

X_test.info()

submission = pd.read_csv('./submission.csv', index_col='Id')

submission['Churn'] = bs1.predict_proba(X_test)[:,1]
submission.to_csv('./my_submission.csv')

In [ ]:
# Бустинг. Я решил исключить некоторые признаки, чтобы улучшить процент. Тут уже конечный результат.
num_cols = [
    'ClientPeriod',
    'MonthlySpending'
]

cat_cols = [
    'Sex',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]
bs2 = catboost.CatBoostClassifier(n_estimators=500, 
                                            cat_features=cat_cols, 
                                            learning_rate=0.022)
X_train = data[cat_cols + num_cols]
y_train = data[target_col]
X_train.info()
bs2.fit(X_train, y_train)

In [ ]:
X_test = pd.read_csv('./test.csv')
X_test['TotalSpent'] = X_test['TotalSpent'].replace(' ', 0)
X_test['TotalSpent'] = X_test['TotalSpent'].astype(float)
X_test = X_test[cat_cols + num_cols]

X_test.info()


submission = pd.read_csv('./submission.csv', index_col='Id')
submission['Churn'] = bs2.predict_proba(X_test)[:, 1]
submission.to_csv('./my_submission.csv')

In [ ]:
# Попытка со стекингом с логистичсекой регрессией в конце.
dummy_features = pd.get_dummies(data[cat_cols])
X_train = pd.concat([data[num_cols], dummy_features], axis=1)
y_train = data[target_col]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = pd.read_csv('./test.csv')
X_test['TotalSpent'] = X_test['TotalSpent'].replace(' ', 0)
X_test['TotalSpent'] = X_test['TotalSpent'].astype(float)
dummy_features = pd.get_dummies(X_test[cat_cols])
X_test = pd.concat([X_test[num_cols], dummy_features], axis=1)
scaler.transform(X_test)

meta_features_train = np.zeros((X_train.shape[0], 0))
meta_features_test = np.zeros((X_test.shape[0], 0))
for model in tqdm(models):
    train, test = compute_meta_feature(model, X_train, X_test, y_train, 5)
    meta_features_train = np.append(meta_features_train, train.reshape((train.size, 1)), axis=1)
    meta_features_test = np.append(meta_features_test, test.reshape((test.size, 1)), axis=1)
    
bs3 = LogisticRegression(C=100)
bs3.fit(meta_features_train, y_train)

submission = pd.read_csv('./submission.csv', index_col='Id')
submission['Churn'] = bs3.predict_proba(meta_features_test)[:, 1]
submission.to_csv('./my_submission.csv')

y_train_predicted = stacking_model.predict_proba(meta_features_train)[:, 1]

train_auc = roc_auc_score(y_train, y_train_predicted)
print(train_auc)

In [ ]:
# Попытка со стекингом с бустингом в конце.
dummy_features = pd.get_dummies(data[cat_cols])
X_train = pd.concat([data[num_cols], dummy_features], axis=1)
y_train = data[target_col]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = pd.read_csv('./test.csv')
X_test['TotalSpent'] = X_test['TotalSpent'].replace(' ', 0)
X_test['TotalSpent'] = X_test['TotalSpent'].astype(float)
dummy_features = pd.get_dummies(X_test[cat_cols])
X_test = pd.concat([X_test[num_cols], dummy_features], axis=1)
scaler.transform(X_test)

meta_features_train = np.zeros((X_train.shape[0], 0))
meta_features_test = np.zeros((X_test.shape[0], 0))
for model in tqdm(models):
    train, test = compute_meta_feature(model, X_train, X_test, y_train, 5)
    meta_features_train = np.append(meta_features_train, train.reshape((train.size, 1)), axis=1)
    meta_features_test = np.append(meta_features_test, test.reshape((test.size, 1)), axis=1)
    
bs4 = catboost.CatBoostClassifier(n_estimators=500, learning_rate=0.022)
bs4.fit(meta_features_train, y_train)

submission = pd.read_csv('./submission.csv', index_col='Id')
submission['Churn'] = bs4.predict_proba(meta_features_test)[:, 1]
submission.to_csv('./my_submission.csv')

y_train_predicted = stacking_model.predict_proba(meta_features_train)[:, 1]

train_auc = roc_auc_score(y_train, y_train_predicted)
print(train_auc)

**Лучшей моделью оказалась вторая. Итоговый результат 0.85223.**

# Kaggle (5 баллов)

Как выставить баллы:

1) 1 >= roc auc > 0.84 это 5 баллов

2) 0.84 >= roc auc > 0.7 это 3 балла

3) 0.7 >= roc auc > 0.6 это 1 балл

4) 0.6 >= roc auc это 0 баллов


Для выполнения задания необходимо выполнить следующие шаги.
* Зарегистрироваться на платформе [kaggle.com](kaggle.com). Процесс выставления оценок будет проходить при подведении итогового рейтинга. Пожалуйста, укажите во вкладке Team -> Team name свои имя и фамилию в формате Имя_Фамилия (важно, чтобы имя и фамилия совпадали с данными на Stepik).
* Обучить модель, получить файл с ответами в формате .csv и сдать его в конкурс. Пробуйте и экспериментируйте. Обратите внимание, что вы можете выполнять до 20 попыток сдачи на kaggle в день.
* После окончания соревнования отправить в итоговый ноутбук с решением на степик. 
* После дедлайна проверьте посылки других участников по критериям. Для этого надо зайти на степик, скачать их ноутбук и проверить скор в соревновании.